In [18]:
import psycopg2
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import nltk.tokenize as tk
import nltk
import string
import gensim
from gensim import corpora
from langdetect import DetectorFactory
from langdetect import detect
from datetime import datetime

DetectorFactory.seed = 0

In [19]:
## Establishing Connection
try:
    conn = psycopg2.connect('''
    dbname='brew' user='rthakur' 
    host='capstone-brewing.cyol5m7lekm6.us-east-1.rds.amazonaws.com' 
    password='brew'
    ''')
except:
    print "Unable to connect to the database"

In [20]:
##Compiling Documents
beerwords = corpora.Dictionary.load('../../data/beerwords.dict')

cur = conn.cursor()
cur.execute("""
select b.id,
       b.name,
       r.text
from wrk.beer b left join wrk.review r on b.id = r.beer_id
order by 1
""")
rows = cur.fetchall()

reviews = []
for row in rows:
    review = row[2].decode('utf-8')
    name = rows[1]
    name_tokens = nltk.word_tokenize(name.lower())
    try:
        if detect(review)=='en':
            reviews.append(review.lower())
    except:
        continue

In [21]:
## Cleaning and Preprocessing

stop = set(stopwords.words('english'))
additional_exclude_words = [u'beer', u'one', u'like', u'well', u'really', u'very', u'it\x92s', 
                            u'it', u'beers',u'would',u'taste',u'tastes',u'get',u'i\x92m',u'quite', u'i\x92ve',
                            u'bit',u'much',u'good',u'better',u'think',u'first',u'new',u'try', u'updated']
stop |= set(additional_exclude_words)
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    #stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = " ".join([ch for ch in doc.lower().split() if ch not in exclude])
    normalized = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
    tokens = tk.word_tokenize(normalized)
    tokens_with_pos = [x[0] for x in nltk.pos_tag(tokens) if x[1] in ('NN','JJ') and 
                       x[0] not in name_tokens and len(x[0])>2 and x[0] not in stop]
    return tokens_with_pos
#and wordnet.sysnet(term)
reviews_clean = [clean(r) for r in reviews]  

In [22]:
# Creating the term dictionary of our corpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(reviews_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
rev_term_matrix = [dictionary.doc2bow(r) for r in reviews_clean]

In [29]:
print str(datetime.now())
# Creating the object for LDA model
Lda = gensim.models.ldamodel.LdaModel

# Running and Training LDA on the document term matrix.
ldamodel = Lda(rev_term_matrix, num_topics=100, id2word = dictionary, passes=50)
print str(datetime.now())

2017-07-30 01:12:51.893000
2017-07-30 01:49:57.330000


In [30]:
print ldamodel.print_topics(num_topics=5, num_words=5)

[(59, u'0.239*"balance" + 0.099*"doubt" + 0.067*"can\x92t" + 0.054*"great" + 0.039*"malty"'), (144, u'0.382*"floral" + 0.049*"woody" + 0.047*"syrup" + 0.042*"treat" + 0.039*"malt"'), (22, u'0.402*"frothy" + 0.053*"white" + 0.035*"hazelnut" + 0.034*"sweet" + 0.031*"heavy"'), (14, u'0.207*"fantastic" + 0.123*"version" + 0.051*"trappist" + 0.049*"intensity" + 0.045*"flavor"'), (102, u'0.135*"tan" + 0.110*"dark" + 0.082*"head" + 0.072*"fruit" + 0.062*"aroma"')]


## Visualization 

In [31]:
import pyLDAvis.gensim

%matplotlib inline
lda_viz = pyLDAvis.gensim.prepare(ldamodel, rev_term_matrix, dictionary)
pyLDAvis.display(lda_viz)

## Topic Similarity 